In [ ]:
# 필요한 라이브러리 로드
# pandas: 데이터 조작 및 분석용
# os: 파일 시스템 경로 작업용
# xlwings: Excel 파일 제어용
import os
import pandas as pd
import xlwings as xw

In [ ]:
# 'adobe_raw' 디렉토리에서 처리할 원본 파일 탐색
# 'New_Weekly_전체_기획전_Ken'으로 시작하는 파일 사용
for filename in os.listdir('./adobe_raw/'):
    if filename.startswith('New_Weekly_전체_기획전_Ken'):
        whole_campaign_file = './adobe_raw/' + filename
        break

In [ ]:
# xlwings로 Excel 파일을 열고 데이터 읽기
# DRM 보호 CSV 파일을 읽기 위해 Excel 앱을 직접 제어하는 xlwings 사용
# try...finally 구문으로 Excel 앱의 확실한 종료 보장
workbook = None
try:
    workbook = xw.Book(whole_campaign_file)
    sheet1 = workbook.sheets[0].used_range.value
    df_whole_campaign = pd.DataFrame(sheet1)
finally:
    if workbook:
        workbook.app.quit()

In [ ]:
# 컬럼명 변경
column_name = ['WeekNumber',
 'UniqueVisitors',
 'Visits',
 'PageViews',
 'TimeSpentPerVisit',
 'OrderAfterVisit',
 'RevenueAfterVisit',
 'OrderAfterCTA',
 'RevenueAfterCTA']

df_whole_campaign.columns = column_name

In [ ]:
# 리포트 헤더 부분(상위 8행) 제외
df_whole_campaign = df_whole_campaign.iloc[8:]

In [ ]:
# '##############################################' 행 삭제
# boolean indexing 사용이 drop보다 효율적
df_whole_campaign = df_whole_campaign[df_whole_campaign['WeekNumber'] != '##############################################']

In [ ]:
# 'CampaignName' 컬럼 생성
df_whole_campaign['CampaignName'] = df_whole_campaign['WeekNumber']

In [ ]:
# 'CampaignName' 컬럼 정제: 'Week' 포함 행에서 날짜 부분 제거
# 예: 'Week 45 (2024-11-04 ~ 2024-11-10)' -> 'Week 45'
# 반복문 대신 .loc와 .str.contains/.str.split으로 성능 개선 및 경고 해결
week_rows_mask = df_whole_campaign['CampaignName'].str.contains('Week', na=False)
df_whole_campaign.loc[week_rows_mask, 'CampaignName'] = df_whole_campaign.loc[week_rows_mask, 'CampaignName'].str.split(' (', expand=True)[0]

In [ ]:
# 데이터 타입 변경
# 숫자 형태 컬럼으로 변환
# pd.to_numeric(errors='coerce')와 .fillna(0)으로 데이터 변환 오류 방지
numeric_cols = [
    'UniqueVisitors', 'Visits', 'PageViews', 'TimeSpentPerVisit',
    'OrderAfterVisit', 'RevenueAfterVisit', 'OrderAfterCTA', 'RevenueAfterCTA'
]
for col in numeric_cols:
    df_whole_campaign[col] = pd.to_numeric(df_whole_campaign[col], errors='coerce').fillna(0)

# 최종 정수형(integer) 타입 변경
int_cols = [
    'UniqueVisitors', 'Visits', 'PageViews', 'OrderAfterVisit',
    'RevenueAfterVisit', 'OrderAfterCTA', 'RevenueAfterCTA'
]
df_whole_campaign[int_cols] = df_whole_campaign[int_cols].astype(int)

In [ ]:
# csv 파일로 저장
# 불필요한 인덱스 컬럼 제외 (index=False)
# Excel에서 한글 깨짐 방지 (encoding='utf-8-sig')
df_whole_campaign.to_csv('./csv_raw/whole_campaign.csv', index=False, encoding='utf-8-sig')